In [1]:
!pip freeze | grep scikit-learn

"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
!python -V

Python 3.11.5


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

c:\Users\sandra.martinez\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sandra.martinez\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [14]:
y_pred.std() #standard deviation of the trips duration in March (Question 1: 6.24)

6.247488852238703

In [20]:
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = "output.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

#size of the output file (Question 2: 66M)


In [ ]:
#let's turn the notebook into a script (Question 3: jupyter nbconvert --to script starter.ipynb)

In [21]:
#pip install pipenv
#pipenv --version
## pipenv, version 2024.0.1
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
#pipenv --python=3.11.5 
#pipenv install scikit-learn==1.3.0 pandas 

#What's the first hash for the Scikit-Learn dependency? (Question 4: "hash": {"sha256": "c8122227c10016a12c68f3bb3d4e50d0704b0ccf8f0dfa9a97195d4fcf7cdb14"})

The scikit-learn version is 1.3.0.


In [22]:
import numpy
print('The numpy version is {}.'.format(numpy.__version__))

import scipy
print('The scipy version is {}.'.format(scipy.__version__))

#pipenv install scipy==1.11.1 numpy==1.24.3

The numpy version is 1.24.3.
The scipy version is 1.11.1.


In [ ]:
#Script in starter.py
#Run the script for April 2023:
##pipenv shell
##pipenv install pyarrow fastparquet
##python starter.py
#What's the mean predicted duration? (Question 5: Mean predicted duration = 14.292282936862449)

In [ ]:
#Docker container
#Commands in the terminal:
##docker build --platform linux/amd64 -t starter_py .
##docker run --platform linux/amd64 -it starter_py 2023 5
#What's the mean predicted duration for May 2023? (Question 6: Mean predicted duration = 0.19174419265916945)